In [2]:
import pandas as pd
import re
import numpy as np

In [13]:
activity = pd.read_csv('pos_state_collar_2_2018_08_16_precise.csv')
collar = pd.read_csv('9dof_processed_C2.csv')
bracelet = pd.read_csv('ble_processed_C2.csv')
gps = pd.read_csv('gps_processed_C2.csv')
activity['start.time'] = activity['start.time'].apply(lambda x: '{:.0f}'.format(x))
activity['end.time'] = activity['end.times'].apply(lambda x: '{:.0f}'.format(x))
del activity['end.times']
collar.head()
activity.head()

,Actor,Behavior,start.time,end.time
0,ID1,stand up,1534400680081,1534400683981
1,ID1,walk up,1534400684081,1534400689781
2,ID1,walk down,1534400689881,1534400691581
3,ID1,stand down,1534400691681,1534400698281
4,ID1,walk up,1534400698381,1534400704881


In [4]:
collar = collar[collar['TIME'] > 1534395960] 
collar.head()

,TIME,ACC_X,ACC_Y,ACC_Z,GYRO_X,GYRO_Y,GYRO_Z,MAG_X,MAG_Y,MAG_Z
4896,1534395962,8.303095,-4.024728,-2.957529,0.009681,-0.000954,0.003682,-11.400001,-17.900000,-62.700001
4897,1534395962,8.796016,-4.120440,-2.397608,0.018544,-0.007499,0.000818,-11.800000,-18.200001,-62.799999
4898,1534395962,8.872586,-3.967300,-3.005385,0.011317,-0.010227,-0.000818,-12.100000,-16.100000,-63.200001
4899,1534395962,8.585447,-4.058227,-2.742175,0.004363,-0.006818,0.001227,-12.100000,-16.800001,-62.799999
4900,1534395962,8.590233,-4.101298,-2.766103,0.004091,-0.001091,0.002045,-11.900001,-17.200001,-60.900002


In [5]:
gps = gps[gps['TIMESTAMP'] > 1534395960] 
gps.head()

,TIMESTAMP,Latitude,Longitude
2,1534395965,0.295187,36.905240
3,1534395966,0.295188,36.905243
4,1534395966,0.295190,36.905245
5,1534395968,0.295195,36.905248
6,1534395970,0.295197,36.905253


In [6]:
bracelet = bracelet[bracelet['TIME'] > 1534395960] 
bracelet.head()

,TIME,ACC_X,ACC_Y,ACC_Z,GYRO_X,GYRO_Y,GYRO_Z
3260,1534395962,1.564453,-0.877197,-0.750732,2.946565,-6.312977,-1.671756
3261,1534395962,1.563721,-0.876953,-0.751465,2.893130,-6.648855,-1.458015
3262,1534395962,1.566162,-0.868652,-0.752686,2.938931,-6.183206,-1.763359
3263,1534395962,1.565430,-0.870605,-0.757080,3.259542,-6.229008,-1.694656
3264,1534395962,1.562256,-0.874023,-0.759277,3.282443,-6.458015,-1.305344


In [7]:
activity_timeline = pd.DataFrame(columns=['Actor','Behavior','time'])
activity_timeline.head()

,Actor,Behavior,time


In [31]:
# for index, row in activity.iterrows():
#     end_time = int(row['end.time'])
#     time = int(row['start.time'])
#     while time<=end_time:
# #         activity_timeline.loc[index] = row['Actor']
# #         activity_timeline.loc[index] = row['Behavior']
# #         activity_timeline.loc[index] = str(time)
# #         time = time+1
#         print(time)
#         activity_timeline.append({'Actor': row['Actor'] , 'Behavior': row['Behavior'], 'time': str(time)},ignore_index=True)
#         time=time+1
        
    
df = collar.groupby(['TIME'])
df.head()

,TIME,ACC_X,ACC_Y,ACC_Z,GYRO_X,GYRO_Y,GYRO_Z,MAG_X,MAG_Y,MAG_Z
4896,1534395962,8.303095,-4.024728,-2.957529,0.009681,-0.000954,0.003682,-11.400001,-17.900000,-62.700001
4897,1534395962,8.796016,-4.120440,-2.397608,0.018544,-0.007499,0.000818,-11.800000,-18.200001,-62.799999
4898,1534395962,8.872586,-3.967300,-3.005385,0.011317,-0.010227,-0.000818,-12.100000,-16.100000,-63.200001
4899,1534395962,8.585447,-4.058227,-2.742175,0.004363,-0.006818,0.001227,-12.100000,-16.800001,-62.799999
4900,1534395962,8.590233,-4.101298,-2.766103,0.004091,-0.001091,0.002045,-11.900001,-17.200001,-60.900002
4908,1534395963,8.743374,-3.828516,-2.139184,-0.024407,0.034907,-0.000545,-9.700000,-16.100000,-68.000000
4909,1534395963,8.724232,-4.067799,-2.206182,-0.049769,0.043906,-0.002454,-7.900000,-14.100000,-70.099998
4910,1534395963,8.341380,-3.708875,-2.335395,-0.080312,0.050042,-0.005863,-6.700000,-8.900001,-76.400002
4911,1534395963,9.154940,-3.546163,-2.646462,-0.102402,0.089039,-0.005045,-3.000000,3.800000,-86.000000
4912,1534395963,9.245867,-3.215954,-2.325824,-0.092312,0.103902,0.023044,-2.200000,7.600000,-104.000000


In [15]:
activity.head()

,Actor,Behavior,start.time,end.time
0,ID1,stand up,1534400680081,1534400683981
1,ID1,walk up,1534400684081,1534400689781
2,ID1,walk down,1534400689881,1534400691581
3,ID1,stand down,1534400691681,1534400698281
4,ID1,walk up,1534400698381,1534400704881


In [31]:
gps['activity'] = ""

In [42]:
len_act = activity.shape[0]
len_gps = gps.shape[0]
i = 0
index = 0
count = 0
int(activity.iloc[[i]]['start.time'])/1000


while i<len_gps and index<len_act:
    st = int(activity.iloc[[index]]['start.time'])/1000
    et = int(activity.iloc[[index]]['end.time'])/1000
    t = gps.iloc[i]['TIMESTAMP']
#     print(t,st,et)
    if t>et:
        index = index+1
        continue
    elif t<st:
        i = i+1
        continue
    elif t>=st and t<=et:
        count = count +1
        gps.iloc[i,  gps.columns.get_loc('activity')] = str(activity.iloc[index]['Behavior'])
#         print(gps.iloc[i]['activity'])
        i = i +1
        
print(count)

1593


In [51]:
gps.iloc[3224:4875]

,TIMESTAMP,Latitude,Longitude,activity
3224,1534400682,0.346035,36.914653,stand up
3225,1534400683,0.346035,36.914653,stand up
3226,1534400685,0.346037,36.914653,walk up
3227,1534400686,0.346038,36.914652,walk up
3228,1534400689,0.346055,36.914643,walk up
3229,1534400690,0.346060,36.914642,walk down
3230,1534400691,0.346065,36.914642,walk down
3231,1534400692,0.346073,36.914637,stand down
3232,1534400693,0.346080,36.914633,stand down
3233,1534400694,0.346082,36.914630,stand down


In [53]:
gps.to_csv('gps_act_merged.csv', encoding='utf-8', index=False)